### 1. 安装HuggingFace 并下载模型到本地

In [1]:
!pip install huggingface-hub -Uqq

In [8]:
from huggingface_hub import snapshot_download
from pathlib import Path

local_model_path = Path("./LLM_Falcon_model")
local_model_path.mkdir(exist_ok=True)
model_name = "tiiuae/falcon-40b"
commit_hash = "88b92aef6921a843415b6142bd5a906197e3fb0e"

In [9]:
snapshot_download(repo_id=model_name, revision=commit_hash,cache_dir=local_model_path)

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

...

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

'LLM_Falcon_model/models--tiiuae--falcon-40b/snapshots/88b92aef6921a843415b6142bd5a906197e3fb0e'

### 2. 把模型拷贝到S3为后续部署做准备

In [10]:
import sagemaker
from sagemaker import image_uris
import boto3
import os
import time
import json

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

In [11]:
s3_model_prefix = "llm/models/LLM_falcon_model"  # folder where model checkpoint will go
model_snapshot_path = list(local_model_path.glob("**/snapshots/*"))[0]
s3_code_prefix = "LLM-RAG/workshop/LLM_falcon_deploy_code"
print(f"s3_code_prefix: {s3_code_prefix}")
print(f"model_snapshot_path: {model_snapshot_path}")

s3_code_prefix: LLM-RAG/workshop/LLM_falcon_deploy_code
model_snapshot_path: LLM_Falcon_model/models--tiiuae--falcon-40b/snapshots/88b92aef6921a843415b6142bd5a906197e3fb0e


In [12]:
!aws s3 cp --recursive {model_snapshot_path} s3://{bucket}/{s3_model_prefix}

upload: LLM_Falcon_model/models--tiiuae--falcon-40b/snapshots/88b92aef6921a843415b6142bd5a906197e3fb0e/.gitattributes to s3://sagemaker-us-west-2-687912291502/llm/models/LLM_falcon_model/.gitattributes
upload: LLM_Falcon_model/models--tiiuae--falcon-40b/snapshots/88b92aef6921a843415b6142bd5a906197e3fb0e/config.json to s3://sagemaker-us-west-2-687912291502/llm/models/LLM_falcon_model/config.json
upload: LLM_Falcon_model/models--tiiuae--falcon-40b/snapshots/88b92aef6921a843415b6142bd5a906197e3fb0e/generation_config.json to s3://sagemaker-us-west-2-687912291502/llm/models/LLM_falcon_model/generation_config.json
upload: LLM_Falcon_model/models--tiiuae--falcon-40b/snapshots/88b92aef6921a843415b6142bd5a906197e3fb0e/README.md to s3://sagemaker-us-west-2-687912291502/llm/models/LLM_falcon_model/README.md
upload: LLM_Falcon_model/models--tiiuae--falcon-40b/snapshots/88b92aef6921a843415b6142bd5a906197e3fb0e/modelling_RW.py to s3://sagemaker-us-west-2-687912291502/llm/models/LLM_falcon_model/mode

### 3. 模型部署准备（entrypoint脚本，容器镜像，服务配置）

In [13]:
inference_image_uri = (
    f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.3-cu117"
)

print(f"Image going to be used is ---- > {inference_image_uri}")

Image going to be used is ---- > 763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.3-cu117


In [14]:
!mkdir -p LLM_falcon_deploy_code

In [25]:
%%writefile LLM_falcon_deploy_code/model.py
from djl_python import Input, Output
import torch
import logging
import math
import os

from transformers import pipeline, AutoModel, AutoTokenizer
import transformers
import torch

def load_model(properties):
    tensor_parallel = properties["tensor_parallel_degree"]
    model_location = properties['model_dir']
    if "model_id" in properties:
        model_location = properties['model_id']
    logging.info(f"Loading model in {model_location}")
    
    tokenizer = AutoTokenizer.from_pretrained(model_location, trust_remote_code=True)
   
    pipeline = transformers.pipeline(
    "text-generation",
    model=model_location,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto")
    
    return pipeline, tokenizer


pipeline = None
tokenizer = None
generator = None


def handle(inputs: Input):
    global pipeline, tokenizer
    if not pipeline:
        pipeline, tokenizer = load_model(inputs.get_properties())

    if inputs.is_empty():
        return None
    data = inputs.get_as_json()
    
    input_sentences = data["inputs"]
    params = data["parameters"]
   
    response=pipeline(
    input_sentences,
    max_length=params["max_new_tokens"],
    do_sample=params["do_sample"],
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id)
    
    result = {"outputs": response}
    return Output().add_as_json(result)

Overwriting LLM_falcon_deploy_code/model.py


#### Note: option.s3url 需要按照自己的账号进行修改

In [26]:
%%writefile LLM_falcon_deploy_code/serving.properties
engine=Python
option.tensor_parallel_degree=4
option.s3url = s3://sagemaker-us-west-2-687912291502/llm/models/LLM_falcon_model/

Overwriting LLM_falcon_deploy_code/serving.properties


#### 注意: Falcon config 上transformers是4.27.4

In [27]:
%%writefile LLM_falcon_deploy_code/requirements.txt
transformers==4.28.1
einops

Overwriting LLM_falcon_deploy_code/requirements.txt


In [28]:
!rm model.tar.gz
!cd LLM_falcon_deploy_code && rm -rf ".ipynb_checkpoints"
!tar czvf model.tar.gz LLM_falcon_deploy_code

LLM_falcon_deploy_code/
LLM_falcon_deploy_code/model.py
LLM_falcon_deploy_code/requirements.txt
LLM_falcon_deploy_code/serving.properties


In [29]:
s3_code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-us-west-2-687912291502/LLM-RAG/workshop/LLM_falcon_deploy_code/model.tar.gz


### 4. 创建模型 & 创建endpoint

In [30]:
from sagemaker.utils import name_from_base
import boto3

model_name = name_from_base(f"falcon") # Append a timestamp to the provided string
print(model_name)
print(f"Image going to be used is ---- > {inference_image_uri}")

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": inference_image_uri,
        "ModelDataUrl": s3_code_artifact
    },
    
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

falcon-2023-06-08-07-41-30-199
Image going to be used is ---- > 763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.3-cu117
Created Model: arn:aws:sagemaker:us-west-2:687912291502:model/falcon-2023-06-08-07-41-30-199


In [31]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"


endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g5.12xlarge",
            "InitialInstanceCount": 1,
            # "VolumeSizeInGB" : 400,
            # "ModelDataDownloadTimeoutInSeconds": 2400,
            "ContainerStartupHealthCheckTimeoutInSeconds": 15*60,
        },
    ],
)
endpoint_config_response

{'EndpointConfigArn': 'arn:aws:sagemaker:us-west-2:687912291502:endpoint-config/falcon-2023-06-08-07-41-30-199-config',
 'ResponseMetadata': {'RequestId': 'bdc5d5a2-1277-4548-9904-5e09e5693f84',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'bdc5d5a2-1277-4548-9904-5e09e5693f84',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '118',
   'date': 'Thu, 08 Jun 2023 07:41:31 GMT'},
  'RetryAttempts': 0}}

In [32]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

Created Endpoint: arn:aws:sagemaker:us-west-2:687912291502:endpoint/falcon-2023-06-08-07-41-30-199-endpoint


#### 持续检测模型部署进度

In [33]:
import time

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:us-west-2:687912291502:endpoint/falcon-2023-06-08-07-41-30-199-endpoint
Status: InService


### 5. 模型测试

In [ ]:
endpoint_name = "falcon-2023-06-08-07-41-30-199-endpoint"
prompts1 = """
You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 3 results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

Only use the following tables:

CREATE TABLE customer (
	c_customer_sk INTEGER NOT NULL, 
	c_customer_id CHAR(16) NOT NULL, 
	c_current_cdemo_sk INTEGER, 
	c_current_hdemo_sk INTEGER, 
	c_current_addr_sk INTEGER, 
	c_first_shipto_date_sk INTEGER, 
	c_first_sales_date_sk INTEGER, 
	c_salutation CHAR(10), 
	c_first_name CHAR(20), 
	c_last_name CHAR(30), 
	c_preferred_cust_flag CHAR(1), 
	c_birth_day INTEGER, 
	c_birth_month INTEGER, 
	c_birth_year INTEGER, 
	c_birth_country VARCHAR(20), 
	c_login CHAR(13), 
	c_email_address CHAR(50), 
	c_last_review_date CHAR(10), 
	PRIMARY KEY (c_customer_sk)
)ENGINE=InnoDB DEFAULT CHARSET=utf8


CREATE TABLE web_sales (
	ws_sold_date_sk INTEGER, 
	ws_sold_time_sk INTEGER, 
	ws_ship_date_sk INTEGER, 
	ws_item_sk INTEGER NOT NULL, 
	ws_bill_customer_sk INTEGER, 
	ws_bill_cdemo_sk INTEGER, 
	ws_bill_hdemo_sk INTEGER, 
	ws_bill_addr_sk INTEGER, 
	ws_ship_customer_sk INTEGER, 
	ws_ship_cdemo_sk INTEGER, 
	ws_ship_hdemo_sk INTEGER, 
	ws_ship_addr_sk INTEGER, 
	ws_web_page_sk INTEGER, 
	ws_web_site_sk INTEGER, 
	ws_ship_mode_sk INTEGER, 
	ws_warehouse_sk INTEGER, 
	ws_promo_sk INTEGER, 
	ws_order_number INTEGER NOT NULL, 
	ws_quantity INTEGER, 
	ws_wholesale_cost DECIMAL(7, 2), 
	ws_list_price DECIMAL(7, 2), 
	ws_sales_price DECIMAL(7, 2), 
	ws_ext_discount_amt DECIMAL(7, 2), 
	ws_ext_sales_price DECIMAL(7, 2), 
	ws_ext_wholesale_cost DECIMAL(7, 2), 
	ws_ext_list_price DECIMAL(7, 2), 
	ws_ext_tax DECIMAL(7, 2), 
	ws_coupon_amt DECIMAL(7, 2), 
	ws_ext_ship_cost DECIMAL(7, 2), 
	ws_net_paid DECIMAL(7, 2), 
	ws_net_paid_inc_tax DECIMAL(7, 2), 
	ws_net_paid_inc_ship DECIMAL(7, 2), 
	ws_net_paid_inc_ship_tax DECIMAL(7, 2), 
	ws_net_profit DECIMAL(7, 2), 
	PRIMARY KEY (ws_item_sk, ws_order_number)
)ENGINE=InnoDB DEFAULT CHARSET=utf8

Question: I need to know the max sales customer's id in sales report
"""
response_model = smr_client.invoke_endpoint(
            EndpointName=endpoint_name,
            Body=json.dumps(
            {
                "inputs": prompts1,
                "parameters": parameters,
            }
            ),
            ContentType="application/json",
        )

response_model['Body'].read()

In [ ]:
# define payload
prompt = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 3 results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

Only use the following tables:

CREATE TABLE customer (
	c_customer_sk INTEGER NOT NULL, 
	c_customer_id CHAR(16) NOT NULL, 
	c_current_cdemo_sk INTEGER, 
	c_current_hdemo_sk INTEGER, 
	c_current_addr_sk INTEGER, 
	c_first_shipto_date_sk INTEGER, 
	c_first_sales_date_sk INTEGER, 
	c_salutation CHAR(10), 
	c_first_name CHAR(20), 
	c_last_name CHAR(30), 
	c_preferred_cust_flag CHAR(1), 
	c_birth_day INTEGER, 
	c_birth_month INTEGER, 
	c_birth_year INTEGER, 
	c_birth_country VARCHAR(20), 
	c_login CHAR(13), 
	c_email_address CHAR(50), 
	c_last_review_date CHAR(10), 
	PRIMARY KEY (c_customer_sk)
)ENGINE=InnoDB DEFAULT CHARSET=utf8


CREATE TABLE web_sales (
	ws_sold_date_sk INTEGER, 
	ws_sold_time_sk INTEGER, 
	ws_ship_date_sk INTEGER, 
	ws_item_sk INTEGER NOT NULL, 
	ws_bill_customer_sk INTEGER, 
	ws_bill_cdemo_sk INTEGER, 
	ws_bill_hdemo_sk INTEGER, 
	ws_bill_addr_sk INTEGER, 
	ws_ship_customer_sk INTEGER, 
	ws_ship_cdemo_sk INTEGER, 
	ws_ship_hdemo_sk INTEGER, 
	ws_ship_addr_sk INTEGER, 
	ws_web_page_sk INTEGER, 
	ws_web_site_sk INTEGER, 
	ws_ship_mode_sk INTEGER, 
	ws_warehouse_sk INTEGER, 
	ws_promo_sk INTEGER, 
	ws_order_number INTEGER NOT NULL, 
	ws_quantity INTEGER, 
	ws_wholesale_cost DECIMAL(7, 2), 
	ws_list_price DECIMAL(7, 2), 
	ws_sales_price DECIMAL(7, 2), 
	ws_ext_discount_amt DECIMAL(7, 2), 
	ws_ext_sales_price DECIMAL(7, 2), 
	ws_ext_wholesale_cost DECIMAL(7, 2), 
	ws_ext_list_price DECIMAL(7, 2), 
	ws_ext_tax DECIMAL(7, 2), 
	ws_coupon_amt DECIMAL(7, 2), 
	ws_ext_ship_cost DECIMAL(7, 2), 
	ws_net_paid DECIMAL(7, 2), 
	ws_net_paid_inc_tax DECIMAL(7, 2), 
	ws_net_paid_inc_ship DECIMAL(7, 2), 
	ws_net_paid_inc_ship_tax DECIMAL(7, 2), 
	ws_net_profit DECIMAL(7, 2), 
	PRIMARY KEY (ws_item_sk, ws_order_number)
)ENGINE=InnoDB DEFAULT CHARSET=utf8

Question: I need to know the max sales customer's id in sales report

"""

# hyperparameters for llm
payload = {
  "inputs": prompt,
  "parameters": {
    "do_sample": True,
    "top_p": 0.9,
    "temperature": 0.8,
    "max_new_tokens": 1024,
    "repetition_penalty": 1.03,
    "stop": ["\nUser:","<|endoftext|>","</s>"]
  }
}

# send request to endpoint
response = llm.predict(payload)

# print assistant respond
assistant = response[0]["generated_text"][len(prompt):]


## 使用HF model加载

In [51]:
!pip list|grep -i sagemaker

sagemaker                                    2.163.0
sagemaker-pyspark                            1.4.5


In [55]:
# TODO: remove once new version is released
#!pip install git+https://github.com/aws/sagemaker-python-sdk --upgrade
# install latest sagemaker SDK
!pip install sagemaker>=2.161.0
!pip install --upgrade boto3

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 108.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.29.144
    Uninstalling botocore-1.29.144:
      Successfully uninstalled botocore-1.29.144
  Attempting uninstall: boto3
    Found existing installation: boto3 1.26.144
    Uninstalling boto3-1.26.144:
      Successfully uninstalled boto3-1.26.144
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.27.71 requires botocore==1.29.71, but you have botocore 1.29.150 which is incompatible.
awscli 1.27.71 requires PyYAML<5.5,>=3.10, but you have pyyaml 6.0 which is incompatible.
aiobotocore 2.4.2 requires botocore<1.27.60,>=1.27.59, b

最新的sagemaker sdk，没有这个llm image uri的api

In [9]:
import sagemaker
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

# retrieve the llm image uri
llm_image = get_huggingface_llm_image_uri(
  "huggingface",
  version="0.8.2"
)

# print ecr image uri
print(f"llm image uri: {llm_image}")

llm image uri: 763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-tgi-inference:2.0.0-tgi0.8.2-gpu-py39-cu118-ubuntu20.04


In [16]:
import json
from sagemaker.huggingface import HuggingFaceModel

# sagemaker config
instance_type = "ml.g5.12xlarge"
number_of_gpu = 4
health_check_timeout = 600

# TGI config
config = {
  'HF_MODEL_ID': "tiiuae/falcon-40b-instruct", # model_id from hf.co/models
  'SM_NUM_GPUS': json.dumps(number_of_gpu), # Number of GPU used per replica
  'MAX_INPUT_LENGTH': json.dumps(1200),  # Max length of input text
  'MAX_TOTEL_TOKENS': json.dumps(3000),  # Max length of the generation (including input text)
  # 'HF_MODEL_QUANTIZE': "bitsandbytes", # comment in to quantize
}

# create HuggingFaceModel
llm_model = HuggingFaceModel(
  role=role,
  image_uri=llm_image,
  env=config
)



In [17]:
# Deploy model to an endpoint
# https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.Model.deploy
llm = llm_model.deploy(
  initial_instance_count=1,
  instance_type=instance_type,
  # volume_size=400, # If using an instance with local SSD storage, volume_size must be None, e.g. p4 but not p3
  container_startup_health_check_timeout=health_check_timeout, # 10 minutes to be able to load the model
)


------------------!

In [20]:
# define payload
prompt = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 3 results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

Only use the following tables:

CREATE TABLE customer (
	c_customer_sk INTEGER NOT NULL, 
	c_customer_id CHAR(16) NOT NULL, 
	c_current_cdemo_sk INTEGER, 
	c_current_hdemo_sk INTEGER, 
	c_current_addr_sk INTEGER, 
	c_first_shipto_date_sk INTEGER, 
	c_email_address CHAR(50), 
	c_last_review_date CHAR(10), 
	PRIMARY KEY (c_customer_sk)
)ENGINE=InnoDB DEFAULT CHARSET=utf8


CREATE TABLE web_sales (
	ws_sold_date_sk INTEGER, 
	ws_sold_time_sk INTEGER, 
	ws_ship_date_sk INTEGER, 
	ws_item_sk INTEGER NOT NULL, 
	ws_bill_customer_sk INTEGER, 
	ws_bill_cdemo_sk INTEGER, 
	ws_bill_hdemo_sk INTEGER, 
	ws_bill_addr_sk INTEGER, 
	ws_ship_customer_sk INTEGER, 
	ws_ship_cdemo_sk INTEGER, 
	ws_ship_hdemo_sk INTEGER, 
	ws_ship_addr_sk INTEGER, 
	ws_web_page_sk INTEGER, 
	ws_web_site_sk INTEGER, 
	ws_ship_mode_sk INTEGER, 
	ws_warehouse_sk INTEGER, 
	ws_promo_sk INTEGER, 
	ws_order_number INTEGER NOT NULL, 
	ws_quantity INTEGER, 
	ws_wholesale_cost DECIMAL(7, 2), 
	ws_list_price DECIMAL(7, 2), 
	ws_sales_price DECIMAL(7, 2), 
	ws_ext_discount_amt DECIMAL(7, 2), 
	ws_ext_sales_price DECIMAL(7, 2), 
	ws_ext_wholesale_cost DECIMAL(7, 2), 
	ws_ext_list_price DECIMAL(7, 2), 
	ws_ext_tax DECIMAL(7, 2), 
	ws_coupon_amt DECIMAL(7, 2), 
	ws_ext_ship_cost DECIMAL(7, 2), 
	ws_net_paid DECIMAL(7, 2), 
	ws_net_paid_inc_tax DECIMAL(7, 2), 
	ws_net_paid_inc_ship DECIMAL(7, 2), 
	ws_net_paid_inc_ship_tax DECIMAL(7, 2), 
	ws_net_profit DECIMAL(7, 2), 
	PRIMARY KEY (ws_item_sk, ws_order_number)
)ENGINE=InnoDB DEFAULT CHARSET=utf8

Question: I need to know the customer's email address of max net paid in sale report table 

"""

# hyperparameters for llm
payload = {
  "inputs": prompt,
  "parameters": {
    "do_sample": True,
    "top_p": 0.9,
    "temperature": 0.8,
    "max_new_tokens": 300,
    "repetition_penalty": 1.03,
    "stop": ["\nUser:","<|endoftext|>","</s>"]
  }
}

# send request to endpoint
response = llm.predict(payload)

# print assistant respond
assistant = response[0]["generated_text"][len(prompt):]


In [21]:
print(assistant)

<p>SQLQuery:</p>

<pre><code>SELECT c.c_email_address FROM customer c 
INNER JOIN web_sales w ON c.c_customer_sk = w.ws_bill_customer_sk
WHERE w.ws_net_paid_inc_tax = (
SELECT MAX(ws_net_paid_inc_tax) FROM web_sales
) ORDER BY w.ws_net_paid_inc_tax DESC LIMIT 1
</code></pre>

<p>SQLResult:</p>

<pre><code>c_email_address
----------
someone@email.com
</code></pre>

<p>Answer:</p>

<pre><code>The customer's email address who had the maximum net paid in the sale report table is someone@email.com.
</code></pre>


#### 清除模型Endpoint和config

In [140]:
!aws sagemaker delete-endpoint --endpoint-name chatglm-2023-04-27-05-49-59-117-endpoint

In [141]:
!aws sagemaker delete-endpoint-config --endpoint-config-name chatglm-2023-04-27-05-49-59-117-config